In [1]:
import pandas as pd
import sqlite3

In [8]:
! pip install openpyxl --user


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 3.6 MB/s eta 0:00:00a 0:00:01


## Converting all data into Dataframe

In [9]:
db_path = 'bollywood_movies.sqlite'
conn = sqlite3.connect(db_path)
bollywood_df = pd.read_sql_query("SELECT * FROM movies", conn)
conn.close()
tollywood_df = pd.read_excel('tollywood_movies.xlsx')
regional_df = pd.read_csv('indian_movies.csv')

## Cleaning column names

In [10]:
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

In [11]:
bollywood_df = clean_column_names(bollywood_df)
tollywood_df = clean_column_names(tollywood_df)
regional_df = clean_column_names(regional_df)

In [14]:
regional_df

,unnamed:_0,movieid,title,director,genre,releaseyear,budget_(crores),boxoffice_(crores),rating,duration_(minutes),leadactor,leadactress,language,productioncompany
0,7,MOV008,Sarkar (Tamil),A. R. Murugadoss,Action Thriller,2005.0,NaN,NaN,7.5,173.0,Vijay,Keerthy Suresh,Tamil,Kalaignar TV
1,9,MOV010,2.0 (Tamil),S. Shankar,Science Fiction Action,2018.0,550.0,800.0,6.9,147.0,Rajinikanth,Amy Jackson,Tamil,Lyca Productions
2,21,MOV022,K.G.F: Chapter 1,Prashanth Neel,Action Drama,2018.0,50.0,250.0,7.9,156.0,Yash,Srinidhi Shetty,Kannada,Hombale Films
3,23,MOV024,Avengers: Endgame (Dubbed),"Anthony Russo, Joseph Russo",Superhero,2019.0,NaN,NaN,8.4,181.0,Robert Downey Jr.,Scarlett Johansson,"English (Dubbed in Telugu, Kannada, Hindi)",Marvel Studios
4,25,MOV026,Pailwaan,S. Krishna,Sports Action,2019.0,30.0,55.0,6.8,NaN,Sudeep,Aakanksha Singh,Kannada,RRR Motion Pictures
5,27,MOV028,Yajamana,"V. Harikrishna, P. Kumar",Action Drama,2019.0,20.0,40.0,6.3,164.0,Darshan,Rashmika Mandanna,Kannada,Media House Studio
6,29,MOV030,Gentleman,Jadesh Kumar,Action Thriller,2019.0,10.0,20.0,6.1,145.0,Puneeth Rajkumar,Nishvika Naidu,Kannada,Sri Jagadguru Movies
7,30,MOV031,Saaho,Sujeeth,Action Thriller,2019.0,300.0,450.0,NaN,170.0,Prabhas,Shraddha Kapoor,"Telugu (Dubbed in Hindi, Kannada)",UV Creations
8,31,MOV032,Kavaludaari,Hemanth M. Rao,Thriller,2019.0,5.0,10.0,7.4,118.0,Rishi,Anant Nag,Kannada,PNK Productions
9,32,MOV033,96 (Kannada Dubbed),C. Premkumar,Romantic Drama,2019.0,NaN,NaN,7.9,158.0,Vijay Sethupathi,Trisha,Kannada (Dubbed from Tamil),Madras Enterprises


In [37]:
common_columns = ['movie_name', 'genre', 'boxoffice', 'budget', 'industry']
bollywood_df['industry'] = 'Bollywood'
tollywood_df['industry'] = 'Tollywood'
regional_df['industry'] = 'Regional'

### Merging Columns to make one df

In [38]:
# Standardize columns for merging
# Helper to rename and filter valid dataframes
def standardize_df(df, name):
    rename_map = {
        'title': 'movie_name',
        'boxoffice_(crores)': 'boxoffice',
        'budget_(crores)': 'budget'
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    missing = [col for col in common_columns if col not in df.columns]
    if missing:
        print(f"⚠️ Skipping {name}: missing columns {missing}")
        return None

    return df[common_columns]


In [39]:
bollywood_clean = standardize_df(bollywood_df, "Bollywood")
tollywood_clean = standardize_df(tollywood_df, "Tollywood")
regional_clean = standardize_df(regional_df, "Regional")

In [41]:
frames = [df for df in [bollywood_clean, tollywood_clean, regional_clean] if df is not None]
movies_df = pd.concat(frames, ignore_index=True)

In [46]:
movies_df

,movie_name,genre,boxoffice,budget,industry
1,Kabhi Khushi Kabhie Gham...,Family Drama,100.0,50.0,Bollywood
3,Baahubali 2: The Conclusion,Epic Fantasy Action,1810.0,250.0,Bollywood
5,Bharat,Drama,325.0,100.0,Bollywood
6,Dangal,Biographical Sports Drama,2140.0,70.0,Bollywood
8,PK,Satirical Science Fiction Comedy,792.0,85.0,Bollywood
...,...,...,...,...,...
72,Saaho,Action Thriller,450.0,300.0,Regional
73,Kavaludaari,Thriller,10.0,5.0,Regional
80,K.G.F: Chapter 1,Action Drama,250.0,50.0,Regional
81,Aruvi,Drama,5.0,1.0,Regional


### Cleaning values

In [42]:
def clean_money_columns(df):
    df['boxoffice'] = pd.to_numeric(df['boxoffice'], errors='coerce')
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
    return df

In [ ]:
movies_df = clean_money_columns(movies_df)
movies_df.isna().sum()
movies_df = movies_df.dropna(subset=['boxoffice', 'budget'])
movies_df = movies_df[movies_df['budget'] > 0]  # avoid divide by zero

In [44]:
movies_df

,movie_name,genre,boxoffice,budget,industry
1,Kabhi Khushi Kabhie Gham...,Family Drama,100.0,50.0,Bollywood
3,Baahubali 2: The Conclusion,Epic Fantasy Action,1810.0,250.0,Bollywood
5,Bharat,Drama,325.0,100.0,Bollywood
6,Dangal,Biographical Sports Drama,2140.0,70.0,Bollywood
8,PK,Satirical Science Fiction Comedy,792.0,85.0,Bollywood
...,...,...,...,...,...
72,Saaho,Action Thriller,450.0,300.0,Regional
73,Kavaludaari,Thriller,10.0,5.0,Regional
80,K.G.F: Chapter 1,Action Drama,250.0,50.0,Regional
81,Aruvi,Drama,5.0,1.0,Regional


### Converting data back to excel

In [ ]:
movies_df.to_excel('cleaned_movies_data.xlsx', index=False)